# Improved Asymptotic Iteration Method (IAIM): Poschl-Teller Potential

Main idea, expand by Taylor series the coefficients $\lambda_n$ and $s_n$, evaluated at point $y = y_0$, as: \
\
$ \lambda_n = \sum_{k}  c_n^k (y-y_0)^k $ \
\
$ s_n = \sum_{k} d_n^k (y-y_0)^k  $

The coefficients follow a recursion relation (matrix elements row $i$ column $n$ of matrices $C$ and $D$): \
\
$ c_n^i = (i+1)c_{n-1}^{i+1} + d_{n-1}^i + \sum_k c_0^kc_{n-1}^{i-k} $ \
\
$ d_n^i = (i+1)d_{n-1}^{i+1} \sum_k d_0^k c_{n-1}^{i-k} $

The quantization condition can be written as (for the n-th iteration): \
\
$\qquad \delta_n^0= c_{n-1}^0 - d_{n-1}^0c_n^0 = 0 $

Parameters: \
\
$\qquad \lambda_0 = \dfrac{2y(1-iw)}{1-y^2} \qquad s_0 = \dfrac{1-2iw-2w^2}{2(1-y^2)} $

# IAIM Algorithm

In [94]:
#USEFUL LIBRATIES
import numpy as np
#import matplotlib.pyplot as plt
from sympy import *

#NUMBER OF ITERATIONS TO PERFORM
N = 6

#SYMBOLIC VARIABLES DEFINITION
y = symbols("y", real=True)
w = symbols("\omega")

#INITIAL PARAMETERS
l0 = (2*y*(1-I*w))/(1-y**2)
s0 = (1-2*I*w-2*w**2)/(2*(1-y**2))

We must compute the series expansion of $\lambda_0$ and $s_0$ up until order $i=n$

In [95]:
#FUNCTION THAT GETS COEFFICIENTS AS LIST FROM SERIES EXPANSION
def get_coeff(a):
    
    coeff = []
    coeff = np.append(coeff, a.subs(y,0))
    for i in range(1,N):
        coeff = np.append(coeff, a.coeff(y**i))

    return coeff

In [96]:
l0_series = series(l0,y,0,N).removeO()
l0_series.collect(y)

y**5*(-2*I*\omega + 2) + y**3*(-2*I*\omega + 2) + y*(-2*I*\omega + 2)

In [97]:
s0_series = series(s0,y,0,N).removeO()
s0_series.collect(y)

-\omega**2 - I*\omega + y**4*(-\omega**2 - I*\omega + 1/2) + y**2*(-\omega**2 - I*\omega + 1/2) + 1/2

In [98]:
l0_coeff = get_coeff(l0_series)
s0_coeff = get_coeff(s0_series)

The coefficients of these series expansions are columns of the matrices $C$ and $D$, with $i$ index representing the rows of the column, and $N$ (iteration number) representing the column. Once we have the first column (obtained from $\lambda_0$ and $s_0$), we move on to the next:

In [99]:
C = np.zeros((N,N),dtype=object)
D = np.zeros((N,N),dtype=object)

In [100]:
for i in range(N):
    C[i,0] = l0_coeff[i]
for i in range(N):
    D[i,0] = s0_coeff[i]

In [101]:
pprint(Matrix(C))

⎡       0         0  0  0  0  0⎤
⎢                              ⎥
⎢-2⋅ⅈ⋅\omega + 2  0  0  0  0  0⎥
⎢                              ⎥
⎢       0         0  0  0  0  0⎥
⎢                              ⎥
⎢-2⋅ⅈ⋅\omega + 2  0  0  0  0  0⎥
⎢                              ⎥
⎢       0         0  0  0  0  0⎥
⎢                              ⎥
⎣-2⋅ⅈ⋅\omega + 2  0  0  0  0  0⎦


In [102]:
pprint(Matrix(D))

⎡        2              1               ⎤
⎢- \omega  - ⅈ⋅\omega + ─  0  0  0  0  0⎥
⎢                       2               ⎥
⎢                                       ⎥
⎢           0              0  0  0  0  0⎥
⎢                                       ⎥
⎢        2              1               ⎥
⎢- \omega  - ⅈ⋅\omega + ─  0  0  0  0  0⎥
⎢                       2               ⎥
⎢                                       ⎥
⎢           0              0  0  0  0  0⎥
⎢                                       ⎥
⎢        2              1               ⎥
⎢- \omega  - ⅈ⋅\omega + ─  0  0  0  0  0⎥
⎢                       2               ⎥
⎢                                       ⎥
⎣           0              0  0  0  0  0⎦


In [103]:
for n in range(0,N-1): #COLUMNS
    for i in range(0,N-1): #ROWS
        D[i,n+1] = (i+1)*D[i+1,n]
        C[i,n+1] = (i+1)*C[i+1,n]+D[i,n]
        for k in range(0,i): #SUM INTERIOR
            C[i,n+1] = C[i,n] + C[k,0]*C[i-k,n]
            D[i,n+1] = D[i,n] + D[k,0]*C[i-k,n]

In [104]:
pprint(Matrix(C))

⎡                         2                5                                  
⎢       0         - \omega  - 3⋅ⅈ⋅\omega + ─    -2⋅ⅈ⋅\omega + 2     -2⋅ⅈ⋅\omeg
⎢                                          2                                  
⎢                                                                             
⎢-2⋅ⅈ⋅\omega + 2       -2⋅ⅈ⋅\omega + 2          -2⋅ⅈ⋅\omega + 2               
⎢                                                                             
⎢                                      2                         2            
⎢       0             (-2⋅ⅈ⋅\omega + 2)       2⋅(-2⋅ⅈ⋅\omega + 2)             
⎢                                                                             
⎢-2⋅ⅈ⋅\omega + 2       -2⋅ⅈ⋅\omega + 2          -2⋅ⅈ⋅\omega + 2               
⎢                                                                             
⎢                                      2                         2            
⎢       0             (-2⋅ⅈ⋅\omega + 2)       2⋅(-2⋅

In [105]:
pprint(Matrix(D))

⎡        2              1                                                     
⎢- \omega  - ⅈ⋅\omega + ─                       0                         (-2⋅
⎢                       2                                                     
⎢                                                                             
⎢                                            ⎛        2              1⎞       
⎢           0              (-2⋅ⅈ⋅\omega + 2)⋅⎜- \omega  - ⅈ⋅\omega + ─⎟  2⋅(-2
⎢                                            ⎝                       2⎠       
⎢                                                                             
⎢        2              1                    2              1                 
⎢- \omega  - ⅈ⋅\omega + ─            - \omega  - ⅈ⋅\omega + ─                 
⎢                       2                                   2                 
⎢                                                                             
⎢                                            ⎛      

In [106]:
#SOLVE FOR EACH ITERATION
d = np.zeros(N, dtype=object)
for n in range(0,N-1):
    d[n] = D[n,N-1]*C[n,N-2] - D[n,N-2]*C[n,N-1]
    print("delta_" + str(n+1) + " = " + str(d[n]))

delta_1 = 4*(-2*I*\omega + 2)*(-\omega**2 - I*\omega + 1/2)*(-2*I*\omega + 2*(-2*I*\omega + 2)*(-\omega**2 - I*\omega + 1/2) + 2) - 3*(-2*I*\omega + 2)*(-\omega**2 - I*\omega + 1/2)*(-2*I*\omega + 3*(-2*I*\omega + 2)*(-\omega**2 - I*\omega + 1/2) + 2)
delta_2 = (-2*I*\omega + 2)**2*(-\omega**2 - I*\omega + 1/2)
delta_3 = -(-2*I*\omega + 2)**2*(-\omega**2 - I*\omega + 1/2)
delta_4 = (-2*I*\omega + 2)**2*(-\omega**2 - I*\omega + 1/2)
delta_5 = -(-2*I*\omega + 2)**2*(-\omega**2 - I*\omega + 1/2)


In [107]:
for n in range(0,N-1):
    print("Iteration " + str(n+1))
    sols = solve(d[n],w)
    #print("All solutions: " + "\n" + str(sols) + "\n")
    print("Filtered solutions:")
    for i in range(len(sols)):
        if re(sols[i]) > 0 and  im(sols[i]) < 0:
            print("w_" + str(i+1) + " = " + str(sols[i]))

Iteration 1
Filtered solutions:
w_3 = 1/2 - I/2
Iteration 2
Filtered solutions:
w_3 = 1/2 - I/2
Iteration 3


Filtered solutions:
w_3 = 1/2 - I/2
Iteration 4
Filtered solutions:
w_3 = 1/2 - I/2
Iteration 5
Filtered solutions:
w_3 = 1/2 - I/2
